In [1]:
# Импорт необходимых библиотек
import pandas as pd 
import numpy as np  
import torch  
import os 
import random 
import torch.nn as nn  
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader  
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score 
import json 
from transformers import AutoTokenizer, AutoModel  
from tqdm import tqdm 
import time 
from collections import Counter
import re 

In [3]:
# --- Конфигурация и Гиперпараметры ---

# Пути к файлам данных
DATA_FILE = 'MY_DATA.csv' # Основной файл с логами взаимодействий
TRACK_METADATA_FILE = 'tracks.json' # Файл с метаданными треков
TOP_TRACKS_FILE = 'top_tracks.json' # Файл с популярными треками

# Параметры текстовой модели (BERT)
TEXT_MODEL_NAME = 'bert-base-multilingual-cased' # Название предобученной модели BERT
MAX_TEXT_LENGTH = 32 # Максимальная длина последовательности для токенизатора BERT
BERT_BATCH_SIZE = 128 # Размер батча для генерации эмбеддингов BERT

# Размерности эмбеддингов и слоев модели
USER_EMB_DIM = 96 # Размерность эмбеддинга пользователя
TRACK_EMB_DIM = 96 # Размерность эмбеддинга трека
GENRE_EMB_DIM = 48 # Размерность эмбеддинга жанра (после трансформации)
PROJ_TEXT_EMB_DIM = 96 # Размерность проекции текстового эмбеддинга BERT для MLP
INTERACTION_EMB_DIM = 96 # Размерность пространства для взаимодействий признаков

# Параметры MLP (многослойного перцептрона)
HIDDEN_DIMS = [384, 192] # Размеры скрытых слоев MLP
DROPOUT_RATE = 0.45 # Коэффициент Dropout для регуляризации MLP

# Параметры обучения
TRAIN_BATCH_SIZE = 256 # Размер батча для обучения
VAL_BATCH_SIZE = 512 # Размер батча для валидации
REC_BATCH_SIZE = 1024 # Размер батча для генерации рекомендаций
NUM_EPOCHS = 15 # Максимальное количество эпох обучения
LEARNING_RATE = 5e-5 # Скорость обучения для оптимизатора AdamW
WEIGHT_DECAY = 5e-5 # Коэффициент L2-регуляризации 
PATIENCE_LIMIT = 3 # Количество эпох без улучшения метрики валидации до ранней остановки
NUM_NEG_SAMPLES = 4 # Количество негативных примеров на каждый позитивный для обучения

# Прочие параметры
SEED = 42 # Зерно для генераторов случайных чисел для воспроизводимости
USE_AMP = True # Использовать ли Automatic Mixed Precision (AMP) для ускорения на GPU

# Формирование имен файлов для сохранения результатов
MODEL_SUFFIX = f"interaction_model_v3.2_bert_{TEXT_MODEL_NAME.replace('/', '_')}_negs{NUM_NEG_SAMPLES}_amp{USE_AMP}"
BEST_MODEL_PATH = f"best_{MODEL_SUFFIX}.pth" 
RECOMMENDATIONS_FILE = f"user_recommendations_{MODEL_SUFFIX}_top100.json"
TEXT_EMBEDDING_FILE_PATH = f"track_text_embeddings_{TEXT_MODEL_NAME.replace('/', '_')}.pth" 
LEARNED_USER_EMB_FILE = f"learned_user_embeddings_{MODEL_SUFFIX}.pth" 
LEARNED_TRACK_EMB_FILE = f"learned_track_embeddings_{MODEL_SUFFIX}.pth" 
USER_TOP_GENRES_FILE = f"user_top_genre_ids_{MODEL_SUFFIX}.json" 
GENRE_TOP_TRACKS_FILE = f"genre_id_top_popular_tracks_{MODEL_SUFFIX}.json" 

In [5]:
# --- Инициализация и Настройка Окружения ---

# Установка зерна для воспроизводимости
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Определение устройства (CPU, CUDA GPU, MPS GPU) и настройка AMP
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED) # Установка зерна для всех GPU

elif torch.backends.mps.is_available(): # Проверка доступности MPS (для Apple Silicon)
    device = torch.device("mps")
    torch.mps.manual_seed(SEED)
    USE_AMP = False # AMP не полностью поддерживается на MPS, отключаем
    print("Warning: MPS device detected. Disabling AMP.")
else:
    device = torch.device("cpu")
    USE_AMP = False # AMP работает только на CUDA GPU
    print("Warning: CUDA not found. Disabling AMP.")

print(f"Using device: {device}")
print(f"AMP enabled: {USE_AMP}")
print(f"Text model: {TEXT_MODEL_NAME}")
print(f"Model Params -> UserEmb: {USER_EMB_DIM}, TrackEmb: {TRACK_EMB_DIM}, HiddenDims: {HIDDEN_DIMS}, Dropout: {DROPOUT_RATE}")
print(f"Training Params -> LR: {LEARNING_RATE}, WeightDecay: {WEIGHT_DECAY}, Patience: {PATIENCE_LIMIT}")


Using device: mps
AMP enabled: False
Text model: bert-base-multilingual-cased
Model Params -> UserEmb: 96, TrackEmb: 96, HiddenDims: [384, 192], Dropout: 0.45
Training Params -> LR: 5e-05, WeightDecay: 5e-05, Patience: 3


In [7]:
# --- Загрузка и Предобработка Данных ---
try:
    # Загрузка данных из CSV файла
    df = pd.read_csv(DATA_FILE)
    # Список столбцов для удаления (если они существуют)
    cols_to_drop = ['message', 'latency', 'recommendation', 'experiments','rnd', 'user_id', 'item_id', 'rating']
    existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
    # Удаление столбцов
    df.drop(existing_cols_to_drop, axis=1, inplace=True, errors='ignore')

except FileNotFoundError:
    # Обработка ошибки, если файл не найден
    print(f"Ошибка: Файл {DATA_FILE} не найден.")
    exit() # Завершение скрипта
except Exception as e:
    # Обработка других ошибок при загрузке
    print(f"Ошибка при загрузке {DATA_FILE}: {e}")
    exit() # Завершение скрипта

# Проверка наличия обязательных столбцов
required_cols = ['time', 'duration', 'genre', 'artist', 'album', 'title', 'pop', 'user', 'track', 'timestamp']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"Ошибка: Отсутствуют столбцы: {missing_cols}")
    exit()

print("Handling missing values...")
# Заполнение пропусков в текстовых столбцах значением 'Unknown'
df.fillna({'artist': 'Unknown', 'album': 'Unknown', 'title': 'Unknown'}, inplace=True)
# Заполнение пропусков в столбце 'genre' значением '()'
df.fillna({'genre': '()'}, inplace=True)

print("Parsing genre IDs from string...")
# Функция для извлечения списка ID жанров из строки вида '(1, 23, 45)'
def parse_genre_ids_from_string(genre_str):
    genre_ids = []
    if isinstance(genre_str, str):
        # Очистка строки от скобок и пробелов по краям
        cleaned_str = genre_str.strip().strip('()')
        if cleaned_str:
            # Поиск всех чисел в строке
            found_numbers = re.findall(r'\d+', cleaned_str)
            for num_str in found_numbers:
                try:
                    # Конвертация найденных строк в целые числа
                    genre_ids.append(int(num_str))
                except ValueError:
                    # Предупреждение, если конвертация не удалась
                    print(f"Предупреждение: Не удалось сконвертировать ID жанра '{num_str}' из строки '{genre_str}'. Пропускается.")
    return genre_ids

# Применение функции парсинга к столбцу 'genre' и создание нового столбца 'genre_id_list'
df['genre_id_list'] = df['genre'].apply(parse_genre_ids_from_string)

# Удаление строк с критическими пропущенными значениями (NaN)
initial_rows = len(df)
df.dropna(subset=['time', 'duration', 'pop', 'user', 'track', 'timestamp'], inplace=True)
print(f"Removed {initial_rows - len(df)} rows with critical NaNs.")

# Создание целевой переменной 'target': 1 если прослушано >= 70%, иначе 0
df['target'] = (df['time'] >= 0.7).astype(int)
print(f"Target distribution:\n{df['target'].value_counts(normalize=True)}")

print("Processing features for the model...")
mlb = MultiLabelBinarizer()

# Преобразование списка ID жанров в бинарную матрицу признаков
genre_features_np = mlb.fit_transform(df['genre_id_list'].tolist())
num_genres = genre_features_np.shape[1] # Количество уникальных жанров
print(f"Found {num_genres} unique genre IDs for model input.")

# Инициализация MinMaxScaler для масштабирования числовых признаков
scaler = MinMaxScaler()
numeric_features_np = scaler.fit_transform(df[['pop', 'duration']].astype(float))
num_numeric_features = numeric_features_np.shape[1] # Количество числовых признаков

cols_to_drop_model_features = ['genre']
df.drop(cols_to_drop_model_features, axis=1, inplace=True, errors='ignore')

Handling missing values...
Parsing genre IDs from string...
Removed 0 rows with critical NaNs.
Target distribution:
0    0.787289
1    0.212711
Name: target, dtype: float64
Processing features for the model...
Found 91 unique genre IDs for model input.


In [9]:
# --- Генерация Текстовых Эмбеддингов (BERT) ---
start_time_bert = time.time()

# Выбор уникальных треков с их метаданными (исполнитель, альбом, название)
unique_tracks_df = df[['track', 'artist', 'album', 'title']].drop_duplicates(subset=['track']).reset_index(drop=True)
unique_tracks_df['combined_text'] = unique_tracks_df['artist'] + " [SEP] " + unique_tracks_df['album'] + " [SEP] " + unique_tracks_df['title']
unique_texts = unique_tracks_df['combined_text'].tolist() # Список уникальных текстовых описаний
unique_track_ids_list = unique_tracks_df['track'].tolist() # Список соответствующих ID треков
n_unique_texts = len(unique_texts)
print(f"Found {n_unique_texts} unique tracks for embedding generation.")

# Загрузка токенизатора и модели BERT
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)
bert_model = AutoModel.from_pretrained(TEXT_MODEL_NAME).to(device) 
bert_model.eval()

unique_embeddings_list = [] 
print(f"Generating embeddings in batches of {BERT_BATCH_SIZE}...")

# Генерация эмбеддингов батчами
with torch.no_grad():
    for i in tqdm(range(0, n_unique_texts, BERT_BATCH_SIZE), desc="Embedding Unique Tracks"):
        batch_texts = unique_texts[i : i + BERT_BATCH_SIZE] # Выбор текстов для текущего батча
        try:
            # Токенизация текстов
            inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=MAX_TEXT_LENGTH, return_tensors='pt').to(device)
            with torch.cuda.amp.autocast(enabled=(USE_AMP and device.type == 'cuda')):
                outputs = bert_model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu()
            unique_embeddings_list.append(cls_embeddings)
            del inputs, outputs, cls_embeddings
            if device.type != 'cpu':
                torch.cuda.empty_cache() if device.type == 'cuda' else torch.mps.empty_cache() if device.type == 'mps' else None
        except Exception as e:
            print(f"Error processing batch {i}: {e}")

# Объединение эмбеддингов из всех батчей в один тензор
unique_text_embeddings_tensor = torch.cat(unique_embeddings_list, dim=0)
bert_output_dim = unique_text_embeddings_tensor.shape[1]
print(f"Generated {unique_text_embeddings_tensor.shape[0]} embeddings, dim={bert_output_dim}.")

# Освобождение памяти
del bert_model, unique_tracks_df, unique_texts, unique_embeddings_list

end_time_bert = time.time()
print(f"Text embedding generation took {end_time_bert - start_time_bert:.2f}s.")

Found 50000 unique tracks for embedding generation.
Generating embeddings in batches of 128...


Embedding Unique Tracks: 100%|████████████████| 391/391 [03:02<00:00,  2.15it/s]

Generated 50000 embeddings, dim=768.
Text embedding generation took 186.80s.


In [10]:
# --- Сохранение Текстовых Эмбеддингов ---
final_text_embedding_map_for_server = {} # Словарь для хранения эмбеддингов {track_id: embedding_tensor}
num_embeddings = unique_text_embeddings_tensor.shape[0]
num_ids = len(unique_track_ids_list)

# Проверка соответствия количества эмбеддингов и ID треков
if num_embeddings != num_ids:
    print(f"[ERROR] Mismatch btw embeddings & track IDs.")
else:
    print(f"Mapping {num_embeddings} text embeddings...")
    # Заполнение словаря
    for i in tqdm(range(num_embeddings), desc="Mapping Text Embeddings"):
        final_text_embedding_map_for_server[unique_track_ids_list[i]] = unique_text_embeddings_tensor[i].cpu().clone()

    print(f"Saving text embedding map to: {TEXT_EMBEDDING_FILE_PATH}")
    try:
        torch.save(final_text_embedding_map_for_server, TEXT_EMBEDDING_FILE_PATH)
        print(f"Saved text embedding map.")
    except Exception as e:
        print(f"[ERROR] Failed saving text embedding file: {e}")

Mapping 50000 text embeddings...


Mapping Text Embeddings: 100%|████████| 50000/50000 [00:00<00:00, 200314.25it/s]


Saving text embedding map to: track_text_embeddings_bert-base-multilingual-cased.pth
Saved text embedding map.


In [11]:
# --- Подготовка Данных для Модели (Индексация, Словари Признаков, Разделение) ---

# Получение всех уникальных ID треков из основного DataFrame
all_track_ids = df['track'].unique()
print(f"Found {len(all_track_ids)} unique tracks.")

# Создание отображений (кодировщиков) из ID пользователя/трека в непрерывные индексы (0, 1, 2, ...)
user_encoder = {user: idx for idx, user in enumerate(df['user'].unique())}
track_encoder = {track: idx for idx, track in enumerate(all_track_ids)}
# Создание обратных отображений (декодеров) из индекса в ID
user_decoder = {idx: user for user, idx in user_encoder.items()}
track_decoder = {idx: track for track, idx in track_encoder.items()}

num_users = len(user_encoder) # Общее количество уникальных пользователей
num_tracks = len(track_encoder) # Общее количество уникальных треков
print(f"Users: {num_users}, Tracks: {num_tracks}")

# Добавление столбцов с индексами пользователя и трека в DataFrame
df['user_idx'] = df['user'].map(user_encoder)
df['track_idx'] = df['track'].map(track_encoder)

if df['user_idx'].isnull().any() or df['track_idx'].isnull().any():
    print("Warning: Nulls after mapping.")
    df.dropna(subset=['user_idx', 'track_idx'], inplace=True)

print("Creating user positive items lookup...")
# Создание словаря, где ключ - user_idx, значение - set() из track_idx, с которыми пользователь взаимодействовал положительно (target=1)
user_pos_items = {}
required_cols_for_pos = ['target', 'user_idx', 'track_idx']
if not all(col in df.columns for col in required_cols_for_pos):
    print(f"Error: Missing {required_cols_for_pos} in df.")
    exit()
else:
    try:
        user_pos_items = df[df['target'] == 1].groupby('user_idx')['track_idx'].apply(set).to_dict()
        print(f"Created user_pos_items map for {len(user_pos_items)} users.")
    except Exception as e:
        print(f"Error creating user_pos_items: {e}")
        exit()

print("Creating feature lookups by track_idx...")
text_emb_idx_map = {} # {track_idx: text_embedding_tensor}
if 'final_text_embedding_map_for_server' in locals(): # Проверяем, существует ли словарь с эмбеддингами
    for original_id, emb in final_text_embedding_map_for_server.items():
        if original_id in track_encoder: 
            text_emb_idx_map[track_encoder[original_id]] = emb
    print(f"Created text_emb_idx_map with {len(text_emb_idx_map)} entries.")
else:
    print("Warning: Text embedding map not found.") 

# Запасной вариант для размерности BERT, если она не определилась ранее
if not text_emb_idx_map and 'bert_output_dim' not in locals():
    bert_output_dim = 768 # Стандартная размерность для bert-base

genre_idx_map = {} # {track_idx: genre_features_tensor}
numeric_idx_map = {} # {track_idx: numeric_features_tensor}

# Создаем вспомогательный словарь для связи track_idx с индексом строки в исходном df,
# чтобы корректно сопоставить признаки из numpy массивов (genre_features_np, numeric_features_np)
track_idx_to_row_index = {}
for idx, track_id in enumerate(df['track']): # Итерируемся по столбцу track в df
     if track_id in track_encoder: # Если трек есть в нашем кодировщике
          track_idx = track_encoder[track_id]
          if track_idx not in track_idx_to_row_index:
              track_idx_to_row_index[track_idx] = df.index[idx] 
              
print("Mapping genre and numeric features...")
# Заполнение словарей признаков по track_idx
for track_idx, row_idx in tqdm(track_idx_to_row_index.items(), desc="Mapping Features"):
     if row_idx < len(genre_features_np):
         # Берем соответствующий вектор жанров, конвертируем в тензор
         genre_idx_map[track_idx] = torch.tensor(genre_features_np[row_idx], dtype=torch.float)
     if row_idx < len(numeric_features_np):
         # Берем соответствующий вектор числовых признаков, конвертируем в тензор
         numeric_idx_map[track_idx] = torch.tensor(numeric_features_np[row_idx], dtype=torch.float)

print("Splitting data...")
# Выделение только положительных взаимодействий для разделения на выборки
df_pos = df[df['target'] == 1][['user_idx', 'track_idx']].copy()
print(f"Positive interactions for splitting: {len(df_pos)}")

if len(df_pos) < 10:
    print("Error: Not enough positive interactions.")
    exit()

# Получение индексов строк с положительными взаимодействиями
pos_indices = df_pos.index.values

# Разделение индексов на обучающую (80%), валидационную (10%) и тестовую (10%) выборки
try:
    # Пытаемся использовать стратификацию по пользователю для сохранения пропорций пользователей в выборках
    train_pos_indices, temp_pos_indices = train_test_split(pos_indices, test_size=0.2, random_state=SEED, stratify=df_pos.loc[pos_indices, 'user_idx'])
    val_pos_indices, test_pos_indices = train_test_split(temp_pos_indices, test_size=0.5, random_state=SEED, stratify=df_pos.loc[temp_pos_indices, 'user_idx'])
except ValueError as e:
    # Если стратификация не удалась (например, слишком мало примеров для некоторых пользователей)
    print(f"Warning: Stratification failed ({e}).")
    # Выполняем обычное разделение
    train_pos_indices, temp_pos_indices = train_test_split(pos_indices, test_size=0.2, random_state=SEED)
    val_pos_indices, test_pos_indices = train_test_split(temp_pos_indices, test_size=0.5, random_state=SEED)

# Создание DataFrame для каждой выборки на основе разделенных индексов
train_pos_data = df_pos.loc[train_pos_indices]
val_pos_data = df_pos.loc[val_pos_indices]
test_pos_data = df_pos.loc[test_pos_indices]
print(f"Split -> Train: {len(train_pos_data)}, Val: {len(val_pos_data)}, Test: {len(test_pos_data)}")

Found 50000 unique tracks.
Users: 10000, Tracks: 50000
Creating user positive items lookup...
Created user_pos_items map for 10000 users.
Creating feature lookups by track_idx...
Created text_emb_idx_map with 50000 entries.
Mapping genre and numeric features...


Mapping Features: 100%|███████████████| 50000/50000 [00:00<00:00, 126627.46it/s]


Splitting data...
Positive interactions for splitting: 1277685
Split -> Train: 1022148, Val: 127768, Test: 127769


In [12]:
# --- Создание Датасетов и Загрузчиков Данных ---

# Определение класса Датасета
class MyDataset(Dataset):
    def __init__(self, positive_data, user_pos_items_map, all_item_indices_set,
                 genre_map, numeric_map, text_emb_map,
                 num_neg_samples, num_genres, num_numeric, text_emb_dim):
        """
        Args:
            positive_data (pd.DataFrame): DataFrame с положительными взаимодействиями (user_idx, track_idx).
            user_pos_items_map (dict): Словарь {user_idx: set(positive_track_idx)}.
            all_item_indices_set (set): Множество всех уникальных track_idx.
            genre_map (dict): Словарь {track_idx: genre_features_tensor}.
            numeric_map (dict): Словарь {track_idx: numeric_features_tensor}.
            text_emb_map (dict): Словарь {track_idx: text_embedding_tensor}.
            num_neg_samples (int): Количество негативных примеров на один позитивный.
            num_genres (int): Общее количество уникальных жанров (размерность вектора).
            num_numeric (int): Количество числовых признаков.
            text_emb_dim (int): Размерность текстового эмбеддинга.
        """
        super().__init__()
        self.positive_data = positive_data
        self.user_pos_items_map = user_pos_items_map
        self.all_item_indices = list(all_item_indices_set)
        self.num_items = len(self.all_item_indices)
        self.genre_map = genre_map
        self.numeric_map = numeric_map
        self.text_emb_map = text_emb_map
        self.num_neg_samples = num_neg_samples

        # Создание тензоров по умолчанию для случаев, когда признак отсутствует для трека
        self.default_genre = torch.zeros(num_genres, dtype=torch.float)
        self.default_numeric = torch.zeros(num_numeric, dtype=torch.float)
        # Проверка и установка размерности текстового эмбеддинга
        if text_emb_dim is None:
             print("Error: text_emb_dim is None in Dataset init! Using fallback 768.")
             text_emb_dim = 768 # Запасное значение
        self.default_text_emb = torch.zeros(text_emb_dim, dtype=torch.float)

        # Генерация списка всех сэмплов (положительных + отрицательных)
        self.samples = self._generate_samples()
        if not self.samples:
            print("Warning: Generated sample list is empty!")


    def _generate_samples(self):
        """Генерирует список сэмплов (user_idx, item_idx, target)."""
        print("Generating samples with negatives...")
        samples = []
        if self.positive_data.empty:
            print("Warning: positive_data is empty, cannot generate samples.")
            return samples

        # Итерация по положительным примерам
        for _, row in tqdm(self.positive_data.iterrows(), total=len(self.positive_data), desc="Generating samples"):
            u_idx, i_pos_idx = row['user_idx'], row['track_idx']
            # Добавление положительного примера
            samples.append((u_idx, i_pos_idx, 1.0)) # target = 1.0

            # Получение множества положительных треков для данного пользователя
            user_positive_set = self.user_pos_items_map.get(u_idx, set())
            neg_count = 0
            attempts = 0
            # Генерация негативных примеров
            while neg_count < self.num_neg_samples and attempts < self.num_neg_samples * 10: 
                # Случайный выбор трека из всех возможных
                i_neg_idx = random.choice(self.all_item_indices)
                # Проверка, что выбранный трек не является положительным для пользователя
                if i_neg_idx not in user_positive_set:
                    # Добавление негативного примера
                    samples.append((u_idx, i_neg_idx, 0.0)) # target = 0.0
                    neg_count += 1
                attempts += 1 # Увеличение счетчика попыток

        print(f"Generated {len(samples)} total samples.")
        return samples

    def __len__(self):
        """Возвращает общее количество сэмплов в датасете."""
        if hasattr(self, 'samples'):
            return len(self.samples)
        else:
            print("Error: self.samples attribute not found in __len__!")
            return 0

    def __getitem__(self, idx):
        """Возвращает один сэмпл (словарь с тензорами) по индексу."""
        if not hasattr(self, 'samples') or not self.samples:
             raise IndexError("Dataset samples are not available or empty.")
        if idx >= len(self.samples):
             raise IndexError(f"Index {idx} out of range for dataset with length {len(self.samples)}")

        # Получение user_idx, item_idx и target для данного сэмпла
        u_idx, i_idx, target = self.samples[idx]

        # Получение признаков для item_idx из соответствующих словарей
        # Используем .get() с тензором по умолчанию, если ключ отсутствует
        text_emb = self.text_emb_map.get(i_idx, self.default_text_emb)
        genre_features = self.genre_map.get(i_idx, self.default_genre)
        numeric_features = self.numeric_map.get(i_idx, self.default_numeric)

        # Дополнительная проверка типов (на случай ошибок при создании словарей)
        if not isinstance(text_emb, torch.Tensor): text_emb = self.default_text_emb
        if not isinstance(genre_features, torch.Tensor): genre_features = self.default_genre
        if not isinstance(numeric_features, torch.Tensor): numeric_features = self.default_numeric

        # Возвращаем словарь с тензорами
        return {
            'user_id': torch.tensor(u_idx, dtype=torch.long), # Индекс пользователя
            'track_id': torch.tensor(i_idx, dtype=torch.long), # Индекс трека
            'text_emb': text_emb, # Текстовый эмбеддинг
            'genre_features': genre_features, # Вектор жанров
            'numeric_features': numeric_features, # Вектор числовых признаков
            'target': torch.tensor(target, dtype=torch.float) # Целевая переменная (0.0 или 1.0)
        }

all_item_indices_set = set(track_encoder.values())

if 'user_pos_items' not in locals():
    print("Fatal Error: user_pos_items not defined!")
    exit()

# Создание экземпляров датасета для обучения и валидации
train_dataset = MyDataset(train_pos_data, user_pos_items, all_item_indices_set, genre_idx_map, numeric_idx_map, text_emb_idx_map, NUM_NEG_SAMPLES, num_genres, num_numeric_features, bert_output_dim)
val_dataset = MyDataset(val_pos_data, user_pos_items, all_item_indices_set, genre_idx_map, numeric_idx_map, text_emb_idx_map, NUM_NEG_SAMPLES, num_genres, num_numeric_features, bert_output_dim)

# Создание загрузчиков данных (DataLoader) для итерации по батчам
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=device.type == 'cuda') # Перемешивание для обучающей выборки
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=device.type == 'cuda') # Без перемешивания для валидационной

Generating samples with negatives...


Generating samples: 100%|██████████| 1022148/1022148 [00:17<00:00, 59896.72it/s]


Generated 5110740 total samples.
Generating samples with negatives...


Generating samples: 100%|████████████| 127768/127768 [00:02<00:00, 61055.36it/s]

Generated 638840 total samples.


In [17]:
# --- Определение Модели ---
class MyRecommender(nn.Module):
    def __init__(self, num_users, num_tracks, text_emb_dim, genre_dim, numeric_dim,
                 user_emb_dim=USER_EMB_DIM, track_emb_dim=TRACK_EMB_DIM,
                 genre_emb_dim=GENRE_EMB_DIM, proj_text_emb_dim=PROJ_TEXT_EMB_DIM,
                 interaction_emb_dim=INTERACTION_EMB_DIM, hidden_dims=HIDDEN_DIMS,
                 dropout_rate=DROPOUT_RATE):
        """
        Args:
            num_users (int): Общее количество уникальных пользователей.
            num_tracks (int): Общее количество уникальных треков.
            text_emb_dim (int): Размерность входного текстового эмбеддинга (из BERT).
            genre_dim (int): Размерность входного вектора жанров (количество уникальных жанров).
            numeric_dim (int): Количество числовых признаков.
            user_emb_dim (int): Размерность эмбеддинга пользователя.
            track_emb_dim (int): Размерность эмбеддинга трека.
            genre_emb_dim (int): Размерность эмбеддинга жанра после трансформации.
            proj_text_emb_dim (int): Размерность проекции текстового эмбеддинга для MLP.
            interaction_emb_dim (int): Размерность пространства для взаимодействий.
            hidden_dims (list): Список размеров скрытых слоев MLP.
            dropout_rate (float): Коэффициент Dropout.
        """
        super().__init__()
        # Слои эмбеддингов для пользователей и треков
        self.user_embedding = nn.Embedding(num_users, user_emb_dim)
        self.track_embedding = nn.Embedding(num_tracks, track_emb_dim)

        # Слои для проекции эмбеддингов пользователя и трека в пространство взаимодействий
        # Если размерность совпадает, используется nn.Identity() (нет проекции)
        self.proj_user = nn.Linear(user_emb_dim, interaction_emb_dim) if user_emb_dim != interaction_emb_dim else nn.Identity()
        self.proj_track = nn.Linear(track_emb_dim, interaction_emb_dim) if track_emb_dim != interaction_emb_dim else nn.Identity()

        # Слои для проекции текстовых эмбеддингов и жанров в пространство взаимодействий
        self.proj_text_inter = nn.Linear(text_emb_dim, interaction_emb_dim)
        self.proj_genre_inter = nn.Linear(genre_dim, interaction_emb_dim) 

        # Линейные слои (MLP) для трансформации текстовых эмбеддингов и жанров перед подачей в основной MLP
        self.text_projection_mlp = nn.Linear(text_emb_dim, proj_text_emb_dim)
        self.genre_transform_mlp = nn.Linear(genre_dim, genre_emb_dim)

        # Расчет размерности входа для основного MLP
        # Складываются размерности всех конкатенируемых векторов
        mlp_input_dim = (user_emb_dim + track_emb_dim + proj_text_emb_dim +
                         genre_emb_dim + numeric_dim + interaction_emb_dim * 3) 
        print(f"Model - MLP Input Dim: {mlp_input_dim}")

        layers = []
        current_dim = mlp_input_dim
        for h_dim in hidden_dims: 
            layers.append(nn.Linear(current_dim, h_dim)) # Линейный слой
            layers.append(nn.LayerNorm(h_dim)) # Нормализация слоя
            layers.append(nn.GELU()) # Функция активации GELU
            layers.append(nn.Dropout(dropout_rate)) # Dropout
            current_dim = h_dim # Обновление текущей размерности

        # Выходной слой MLP (1 нейрон для предсказания логита)
        layers.append(nn.Linear(current_dim, 1))
        # Объединение всех слоев в последовательную модель
        self.mlp = nn.Sequential(*layers)

        # Инициализация весов модели
        self._init_weights()

    def _init_weights(self):
        """Инициализирует веса Embedding и Linear слоев с помощью Xavier Uniform."""
        for module in self.modules(): 
            if isinstance(module, nn.Embedding):
                nn.init.xavier_uniform_(module.weight)
            elif isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None: 
                    nn.init.zeros_(module.bias)

    def forward(self, user_ids, track_ids, text_emb, genre_features, numeric_features):
        """
        Прямой проход модели.

        Args:
            user_ids (torch.Tensor): Тензор с индексами пользователей.
            track_ids (torch.Tensor): Тензор с индексами треков.
            text_emb (torch.Tensor): Тензор с текстовыми эмбеддингами треков.
            genre_features (torch.Tensor): Тензор с бинарными векторами жанров.
            numeric_features (torch.Tensor): Тензор с числовыми признаками.

        Returns:
            torch.Tensor: Тензор с логитами предсказаний (размер батча).
        """
        # 1. Получение эмбеддингов пользователей и треков
        user_emb = self.user_embedding(user_ids)
        track_emb = self.track_embedding(track_ids)

        # 2. Трансформация жанров и текстовых эмбеддингов для MLP
        # Применяется линейный слой и ReLU
        genre_emb_mlp = F.relu(self.genre_transform_mlp(genre_features))
        projected_text_emb_mlp = F.relu(self.text_projection_mlp(text_emb))

        # 3. Проекция эмбеддингов в пространство взаимодействий
        # Применяется линейный слой проекции и ReLU
        user_inter = F.relu(self.proj_user(user_emb))
        track_inter = F.relu(self.proj_track(track_emb))
        text_inter = F.relu(self.proj_text_inter(text_emb))

        # 4. Расчет взаимодействий признаков (поэлементное умножение)
        interaction_ut = user_inter * track_inter # Пользователь-Трек
        interaction_ui = user_inter * text_inter # Пользователь-Текст
        interaction_ti = track_inter * text_inter # Трек-Текст

        # 5. Конкатенация всех признаков и взаимодействий для MLP
        combined = torch.cat([
            user_emb,                  
            track_emb,                 
            projected_text_emb_mlp,    
            genre_emb_mlp,             
            numeric_features,          
            interaction_ut,            
            interaction_ui,            
            interaction_ti             
        ], dim=1) 

        # 6. Прогон через MLP и получение логитов
        return self.mlp(combined).squeeze(-1)

In [19]:
# --- Настройка Обучения ---

# Создание экземпляра модели
model = MyRecommender(
    num_users=num_users,
    num_tracks=num_tracks,
    text_emb_dim=bert_output_dim, # Размерность эмбеддинга BERT
    genre_dim=num_genres, # Количество уникальных жанров
    numeric_dim=num_numeric_features # Количество числовых признаков
).to(device)

# Определение оптимизатора (AdamW) и функции потерь (BCEWithLogitsLoss)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
loss_fn = nn.BCEWithLogitsLoss()

# Определение планировщика скорости обучения
# Уменьшает LR, если метрика валидации ('max' - AUC) не улучшается
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=max(1, PATIENCE_LIMIT // 2), factor=0.1, verbose=True)

# Инициализация GradScaler для AMP
scaler = torch.cuda.amp.GradScaler(enabled=(USE_AMP and device.type == 'cuda'))

# Инициализация переменных для отслеживания лучшей модели и ранней остановки
best_val_metric = -float('inf') # Лучшее значение метрики валидации (AUC)
patience_counter = 0 # Счетчик эпох без улучшения

# Запуск таймера обучения
training_start_time = time.time()
print(f"Starting training for up to {NUM_EPOCHS} epochs...")


# --- Цикл Обучения ---
for epoch in range(NUM_EPOCHS):
    epoch_start_time = time.time() # Время начала эпохи
    # --- Фаза Обучения ---
    model.train() # Перевод модели в режим обучения
    train_loss = 0.0 # Суммарные потери на обучающей выборке за эпоху
    # Обертка загрузчика данных в tqdm для прогресс-бара
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]", leave=False)

    for batch in train_loop:
        # Перемещение данных батча на выбранное устройство
        user_ids = batch['user_id'].to(device)
        track_ids = batch['track_id'].to(device)
        text_emb = batch['text_emb'].to(device)
        genre_features = batch['genre_features'].to(device)
        numeric_features = batch['numeric_features'].to(device)
        targets_batch = batch['target'].to(device) 

        # Обнуление градиентов перед обратным проходом
        optimizer.zero_grad(set_to_none=True) 

        # Прямой проход с использованием AMP 
        with torch.cuda.amp.autocast(enabled=(USE_AMP and device.type=='cuda')):
            # Получение логитов от модели
            outputs = model(user_ids, track_ids, text_emb, genre_features, numeric_features)
            # Расчет потерь
            loss = loss_fn(outputs, targets_batch)

        # Обратный проход с использованием GradScaler 
        scaler.scale(loss).backward() # Масштабирование потерь перед обратным проходом
        scaler.step(optimizer) # Шаг оптимизатора (с проверкой градиентов)
        scaler.update() # Обновление масштаба GradScaler

        # Накопление потерь и обновление прогресс-бара
        train_loss += loss.item() # Добавляем значение потерь 
        train_loop.set_postfix(loss=loss.item()) # Отображение текущих потерь в tqdm

    # Расчет средней потери на обучающей выборке за эпоху
    avg_train_loss = train_loss / len(train_loader)

    # --- Фаза Валидации ---
    model.eval() 
    val_loss = 0.0 # Суммарные потери на валидационной выборке
    all_targets = [] # Список для хранения всех целевых значений
    all_preds = [] # Список для хранения всех предсказаний (вероятностей)
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]", leave=False)

    with torch.no_grad(): 
        for batch in val_loop:
            # Перемещение данных батча на устройство
            user_ids = batch['user_id'].to(device)
            track_ids = batch['track_id'].to(device)
            text_emb = batch['text_emb'].to(device)
            genre_features = batch['genre_features'].to(device)
            numeric_features = batch['numeric_features'].to(device)
            targets_batch = batch['target'].to(device)

            with torch.cuda.amp.autocast(enabled=(USE_AMP and device.type=='cuda')):
                outputs = model(user_ids, track_ids, text_emb, genre_features, numeric_features)
                loss = loss_fn(outputs, targets_batch) 

            val_loss += loss.item() 

            # Получение вероятностей с помощью сигмоиды
            preds = torch.sigmoid(outputs).cpu().numpy()
            all_preds.extend(preds) 
            all_targets.extend(targets_batch.cpu().numpy()) 

    # Расчет средней потери на валидационной выборке
    avg_val_loss = val_loss / len(val_loader)
    # Расчет AUC ROC
    try:
        # Могут возникнуть ошибки, если в выборке только один класс
        val_auc = roc_auc_score(all_targets, all_preds)
    except ValueError:
        val_auc = 0.0 # В случае ошибки присваиваем AUC=0
        print("Warning: AUC calculation failed.")

    epoch_end_time = time.time() # Время окончания эпохи
    # Вывод результатов эпохи
    print(f'Epoch {epoch+1}/{NUM_EPOCHS} [{epoch_end_time - epoch_start_time:.2f}s] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val AUC: {val_auc:.4f}')

    # Шаг планировщика LR на основе метрики валидации (AUC)
    scheduler.step(val_auc)

    # --- Проверка Улучшения и Ранняя Остановка ---
    current_metric = val_auc # Текущая метрика для сравнения

    if current_metric > best_val_metric:
        # Если метрика улучшилась
        best_val_metric = current_metric
        # Сохранение состояния модели (весов)
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"*** New best model saved (Val AUC: {best_val_metric:.4f}) ***")
        patience_counter = 0 
    else:
        patience_counter += 1
        print(f"Val AUC didn't improve. Patience: {patience_counter}/{PATIENCE_LIMIT}")

    # Проверка условия для ранней остановки
    if patience_counter >= PATIENCE_LIMIT:
        print(f"Early stopping @ epoch {epoch+1}.")
        break 

training_end_time = time.time() 
print(f"Training finished in {training_end_time - training_start_time:.2f} seconds.")

In [21]:
# --- Извлечение и Сохранение Обученных Эмбеддингов ---
learned_user_embeddings = None
learned_track_embeddings = None

# Проверка существования файла с лучшей моделью
if not os.path.exists(BEST_MODEL_PATH):
    print(f"Error: Best model not found at {BEST_MODEL_PATH}. Cannot extract embeddings.")
else:
    print(f"Loading best model: {BEST_MODEL_PATH}...")
    # Создание экземпляра архитектуры модели
    model_loaded = MyRecommender(num_users=num_users, num_tracks=num_tracks, text_emb_dim=bert_output_dim, genre_dim=num_genres, numeric_dim=num_numeric_features).to(device)
    # Загрузка сохраненных весов
    model_loaded.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device)) # map_location для корректной загрузки на любое устройство
    model_loaded.eval()
    print(f" Loaded.")

    print("Extracting embeddings...")
    try:
        # Извлечение тензоров эмбеддингов из модели
        user_embeddings_tensor = model_loaded.user_embedding.weight.detach().cpu()
        track_embeddings_tensor = model_loaded.track_embedding.weight.detach().cpu()
        print(f"UserEmb Shape: {user_embeddings_tensor.shape}, TrackEmb Shape: {track_embeddings_tensor.shape}")

        # Создание словарей для сохранения эмбеддингов с оригинальными ID
        learned_user_embedding_map = {}
        print("Mapping users...")
        for original_user_id, user_idx in tqdm(user_encoder.items(), desc="Map User Emb"):
            # Проверка, что индекс пользователя не выходит за пределы тензора эмбеддингов
            if user_idx < user_embeddings_tensor.shape[0]:
                learned_user_embedding_map[original_user_id] = user_embeddings_tensor[user_idx]
            else:
                 learned_user_embedding_map[original_user_id] = None 
                
        learned_user_embedding_map = {k:v for k,v in learned_user_embedding_map.items() if v is not None}

        learned_track_embedding_map = {}
        print("Mapping tracks...")
        for original_track_id, track_idx in tqdm(track_encoder.items(), desc="Map Track Emb"):
             if track_idx < track_embeddings_tensor.shape[0]:
                 learned_track_embedding_map[original_track_id] = track_embeddings_tensor[track_idx]

        # Сохранение словарей эмбеддингов
        print(f"Saving user embeddings to {LEARNED_USER_EMB_FILE}...")
        torch.save(learned_user_embedding_map, LEARNED_USER_EMB_FILE)
        print(f" Saved {len(learned_user_embedding_map)} user embeddings.")
        learned_user_embeddings = learned_user_embedding_map 

        print(f"Saving track embeddings to {LEARNED_TRACK_EMB_FILE}...")
        torch.save(learned_track_embedding_map, LEARNED_TRACK_EMB_FILE)
        print(f" Saved {len(learned_track_embedding_map)} track embeddings.")
        learned_track_embeddings = learned_track_embedding_map

    except Exception as e:
        print(f"Error extracting or saving embeddings: {e}")


In [23]:
# --- Генерация Рекомендаций ---
recommendations_list = [] # Список для хранения рекомендаций [{'user': user_id, 'tracks': [track_id1, ...]}, ...]

# Проверка, была ли модель успешно загружена
if 'model_loaded' not in locals() or not hasattr(model_loaded, 'state_dict') or model_loaded is None:
    print("Error: Best model not available or not loaded correctly. Cannot generate recommendations.")
else:
    model_loaded.eval() 

    # Отладочная проверка наличия всех необходимых переменных
    print("Debug: Checking required variables for recommendation generation...")
    required_vars = ['genre_idx_map', 'numeric_idx_map', 'text_emb_idx_map', 'user_pos_items', 'track_encoder', 'user_encoder', 'track_decoder']
    all_vars_available = True
    for var_name in required_vars:
        if var_name not in locals():
            print(f"Debug: Variable '{var_name}' is not in local scope.")
            all_vars_available = False
        elif locals()[var_name] is None:
             print(f"Debug: Variable '{var_name}' is None.")
             all_vars_available = False
        # Проверка на пустоту для словарей и списков
        elif hasattr(locals()[var_name], '__len__') and len(locals()[var_name]) == 0:
             print(f"Debug: Variable '{var_name}' is available but empty.")
        elif hasattr(locals()[var_name], '__len__'):
             print(f"Debug: Variable '{var_name}' is available, size: {len(locals()[var_name])}")
        else:
             print(f"Debug: Variable '{var_name}' is available, type: {type(locals()[var_name])}")

    if not all_vars_available:
         print("Error: Feature maps or encoders/decoders unavailable. Cannot generate recommendations.")
    else:
        print("All required feature maps and encoders/decoders are available.")
        print("Preparing features for all tracks...")

        # Получение списка всех индексов треков
        all_items_internal_indices = list(track_encoder.values())
        num_all_items_for_rec = len(all_items_internal_indices)

        # Проверка наличия ключевых переменных конфигурации
        required_config_vars = ['bert_output_dim', 'num_genres', 'num_numeric_features', 'device', 'USE_AMP', 'REC_BATCH_SIZE']
        if not all(var in locals() for var in required_config_vars):
             print("Fatal Error: Core configuration variables (dimensions, device, batch size, AMP) are not defined.")
        else:
            # Создание тензоров по умолчанию на нужном устройстве
            default_text_emb = torch.zeros(bert_output_dim, dtype=torch.float, device=device)
            default_genre = torch.zeros(num_genres, dtype=torch.float, device=device)
            default_numeric = torch.zeros(num_numeric_features, dtype=torch.float, device=device)

            # Подготовка тензоров с признаками для всех треков
            # Используем .get() со значением по умолчанию, если признак отсутствует
            all_items_text_emb = torch.stack([text_emb_idx_map.get(i, default_text_emb.cpu()).to(device) for i in all_items_internal_indices])
            all_items_genre_feat = torch.stack([genre_idx_map.get(i, default_genre.cpu()).to(device) for i in all_items_internal_indices])
            all_items_numeric_feat = torch.stack([numeric_idx_map.get(i, default_numeric.cpu()).to(device) for i in all_items_internal_indices])
            # Тензор с индексами всех треков
            all_items_internal_indices_tensor = torch.tensor(all_items_internal_indices, dtype=torch.long).to(device)

            # Получение списка оригинальных ID пользователей
            unique_original_user_ids = list(user_encoder.keys())
            num_users_for_rec = len(unique_original_user_ids)

            print(f"Generating Top-100 recommendations for {num_users_for_rec} users...")
            rec_gen_start_time = time.time()

            with torch.no_grad():
                user_loop = tqdm(unique_original_user_ids, desc="Generating Recs per User")

                # Итерация по каждому пользователю
                for original_user_id in user_loop:
                    if original_user_id in user_encoder: # Проверка, что пользователь есть в кодировщике
                        internal_user_idx = user_encoder[original_user_id] # Получаем индекс пользователя
                        # Создаем тензор с индексом пользователя (повторяется для батча)
                        user_idx_tensor = torch.tensor([internal_user_idx], device=device, dtype=torch.long)
                        # Получаем множество треков, с которыми пользователь уже взаимодействовал
                        user_positive_set = user_pos_items.get(internal_user_idx, set())

                        user_scores = [] # Список для хранения скоров (логитов) для этого пользователя
                        item_indices_scored = [] # Список для хранения индексов треков, для которых получены скоры

                        # Итерация по всем трекам батчами
                        for i in range(0, num_all_items_for_rec, REC_BATCH_SIZE):
                            # Выбор батча индексов и признаков треков
                            batch_track_indices = all_items_internal_indices_tensor[i : i + REC_BATCH_SIZE]
                            batch_text_emb = all_items_text_emb[i : i + REC_BATCH_SIZE]
                            batch_genre_feat = all_items_genre_feat[i : i + REC_BATCH_SIZE]
                            batch_numeric_feat = all_items_numeric_feat[i : i + REC_BATCH_SIZE]

                            current_batch_size = len(batch_track_indices)
                            # Расширяем тензор индекса пользователя до размера батча
                            user_idx_tensor_batch = user_idx_tensor.expand(current_batch_size)

                            # Получение скоров от модели 
                            with torch.cuda.amp.autocast(enabled=(USE_AMP and device.type == 'cuda')):
                                scores = model_loaded(
                                    user_ids=user_idx_tensor_batch,
                                    track_ids=batch_track_indices,
                                    text_emb=batch_text_emb,
                                    genre_features=batch_genre_feat,
                                    numeric_features=batch_numeric_feat
                                )

                            # Добавляем скоры и индексы треков в списки
                            user_scores.append(scores.cpu())
                            item_indices_scored.extend(batch_track_indices.cpu().tolist())

                        # Объединение скоров из всех батчей для пользователя
                        all_user_scores = torch.cat(user_scores)
                        # Преобразование списка индексов в numpy массив
                        all_items_indices_scored = np.array(item_indices_scored)

                        # Фильтрация: удаляем треки, с которыми пользователь уже взаимодействовал
                        # Создаем маску: True для треков, которых нет в user_positive_set
                        mask = np.isin(all_items_indices_scored, list(user_positive_set), invert=True)
                        # Применяем маску к скорам и индексам
                        filtered_scores = all_user_scores[mask]
                        filtered_indices = all_items_indices_scored[mask]

                        # Выбор топ-100 рекомендаций
                        if len(filtered_scores) == 0:
                            # Если после фильтрации не осталось треков
                            top_original_track_ids = []
                        else:
                            # Определяем количество рекомендаций (не больше доступного и не больше 100)
                            k = min(100, len(filtered_scores))
                            # Находим индексы k наибольших скоров
                            _, top_relative_indices = torch.topk(filtered_scores, k=k, largest=True)
                            # Получаем соответствующие внутренние индексы треков
                            top_internal_track_indices = filtered_indices[top_relative_indices.numpy()]
                            # Декодируем внутренние индексы в оригинальные ID треков
                            top_original_track_ids = [track_decoder.get(idx, -1) for idx in top_internal_track_indices]
                            # Удаляем возможные ошибки декодирования (-1)
                            top_original_track_ids = [tid for tid in top_original_track_ids if tid != -1]

                        # Добавление рекомендаций для пользователя в общий список
                        recommendations_list.append({'user': original_user_id, 'tracks': top_original_track_ids})

            rec_gen_end_time = time.time()
            print(f"Recommendation generation complete in {rec_gen_end_time - rec_gen_start_time:.2f} seconds.")

In [25]:
# --- Сохранение Рекомендаций ---

# Класс для корректной сериализации numpy и torch типов в JSON
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        # Конвертация numpy int типов в Python int
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, np.int64, np.uint8, np.uint16, np.uint32, np.uint64)):
            return int(obj)
        # Конвертация numpy float типов в Python float
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        # Конвертация numpy массивов в Python list
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        # Конвертация torch тензоров в Python list
        elif isinstance(obj, (torch.Tensor,)):
            return obj.tolist()
        # Конвертация set в list
        elif isinstance(obj, (set,)):
            return list(obj)
        # Стандартное поведение для других типов
        return super().default(obj)

# Функция для сохранения данных в формате JSON Lines (.jsonl)
def save_jsonlines(data, filename):
    print(f"\nSaving recommendations to {filename}...")
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            # Итерация по списку рекомендаций
            for item in tqdm(data, desc="Saving recommendations"):
                try:
                    # Подготовка элемента для записи (гарантируем нужные типы)
                    processed_item = {
                        'user': item.get('user'),
                        'tracks': [track for track in item.get('tracks', [])]
                    }
                    # Явная конвертация user ID, если это numpy integer
                    if isinstance(processed_item['user'], np.integer):
                         processed_item['user'] = int(processed_item['user'])
                    # Явная конвертация track ID, если это numpy integer
                    processed_item['tracks'] = [int(t) if isinstance(t, np.integer) else t for t in processed_item['tracks']]

                    # Сериализация элемента в JSON строку с использованием кастомного энкодера
                    # ensure_ascii=False для корректного сохранения не-ASCII символов (если есть в ID)
                    json_line = json.dumps(processed_item, ensure_ascii=False, cls=NumpyEncoder)
                    # Запись JSON строки в файл с добавлением символа новой строки
                    f.write(json_line + '\n')
                except Exception as e_inner:
                    # Обработка ошибок при сериализации отдельного элемента
                    print(f"Error serializing item for user {item.get('user')}: {e_inner}\nItem: {item}")

        print(f"Recommendations saved to {filename}")
    except Exception as e_outer:
        # Обработка ошибок при записи в файл
        print(f"Error writing file {filename}: {e_outer}")

# Проверка наличия переменной с путем к файлу рекомендаций
if 'RECOMMENDATIONS_FILE' in locals():
     save_jsonlines(recommendations_list, RECOMMENDATIONS_FILE) # Вызов функции сохранения
else:
     print("Error: RECOMMENDATIONS_FILE is not defined. Cannot save recommendations.")

print(f'\n--- Script Finished ---')